# 🏥 LegalMed QnA Demo

In [ ]:
from IPython.display import Image, display

# Document and image path (simulated)
doc_image_path = "../data/sample_documents/sample_scan.jpg"
text_context = '''
The patient was last assessed on March 12, 2024. The evaluation concluded that the injury is minor.
'''

# Display the document image
display(Image(doc_image_path))
print("Context:
", text_context.strip())


In [ ]:
from qa_engine.retriever import simple_retrieval

# Simulated knowledge base
knowledge_base = [
    {"content": "The last assessment was performed on March 12, 2024.", "source": "Form A, Page 2"},
    {"content": "Claimant completed rehabilitation on February 1, 2023.", "source": "Rehab Report"}
]

# Retrieve relevant context
retrieved_context, retrieved_source = simple_retrieval(question, knowledge_base)
print("🔍 Retrieved Context:
", retrieved_context)
print("📄 Source:", retrieved_source)

# Override context for QnA
text_context = retrieved_context


In [ ]:
from qa_engine.qa_pipeline import fill_prompt

question = "When was the last assessment conducted?"
prompt = fill_prompt(question, text_context)

print("🧠 LLM Prompt:")
print(prompt)


In [ ]:
import base64
import json
from PIL import Image
import io

# Simulated image-to-base64
with open(doc_image_path, "rb") as f:
    img = Image.open(f)
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    img_b64 = base64.b64encode(buffer.getvalue()).decode("utf-8")

system_prompt = (
    "You are a professional document analyst specializing in extracting meaningful insights "
    "from images and text. Your responses should be concise, structured, and highly accurate."
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": img_b64
                },
            },
            {
                "type": "text",
                "text": prompt
            }
        ],
    }
]

# Payload body for Bedrock Claude
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "system": system_prompt,
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.7,
    "messages": messages,
    "stop_sequences": ["### END"]
})

print("📦 Payload prepared for Bedrock Claude:")
print(json.dumps(json.loads(body), indent=2))


In [ ]:
def extract_answer(model_output_str):
    try:
        parsed = json.loads(model_output_str)
        return {
            "answer": parsed.get("answer", "Not found"),
            "source": parsed.get("source", "Unknown")
        }
    except Exception as e:
        return {"answer": "Parse error", "source": "N/A"}

# Simulated response from model
mock_response = '{"answer": "March 12, 2024", "source": "Assessment Form, Page 2"}'
result = extract_answer(mock_response)
print("✅ Extracted Result:")
print(result)


In [ ]:
# Test multiple questions with retrieval simulation
questions = [
    "When was the last assessment?",
    "When was the rehab program completed?"
]

for q in questions:
    ctx, src = simple_retrieval(q, knowledge_base)
    prompt = fill_prompt(q, ctx)
    print(f"\n🧠 Question: {q}")
    print("Prompt Sent:")
    print(prompt)
    print("📄 Source:", src)
